In [1]:
!pip install gluonnlp pandas tqdm   
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2

     |████████████████████████████████| 344 kB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 11.3 MB 12.0 MB/s eta 0:00:01
  Using cached Cython-3.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.5 MB)
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=608536 sha256=fdc6fe32c8d75553f85e47141c64e2268736930a7b8420e1142a32fd5747c0b3
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 49.1 MB 47.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 5.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.5 MB 5.5 MB/s eta 0:00:01     |████▊                           | 368 kB 5.5 MB/s eta 0:00:01
     |████████████████████████████████| 758 kB 51.3 MB/s eta 0:00:01
     |████████████████████████████████| 880 kB 32.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 63.0 MB

In [2]:
!pip install ipywidgets
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ts9b8u9o
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-ts9b8u9o
     |████████████████████████████████| 129 kB 5.3 MB/s eta 0:00:01
     |████████████████████████████████| 54.7 MB 28.0 MB/s eta 0:00:01
     |████████████████████████████████| 4.5 MB 73.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.5 MB 73.6 MB/s eta 0:00:01
     |████████████████████████████████| 311 kB 47.5 MB/s eta 0:00:01
     |████████████████████████████████| 73 kB 1.7 MB/s  eta 0:00:01
     |████████████████████████████████| 6.7 MB 60.9 MB/s eta 0:00:01
     |████████████████████████████████| 127 kB 67.1 MB/s eta 0:00:01
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15708 sha256=34498b96452e49dcd230edbb6ad2c8bee9234fb6b27f3c934c60778e7ee1d794
  Stored in directory: /tmp/pip-ephem-wheel-cache-bj5b7twx/wheels/d3/68/ca/334747dfb0383

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [2]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [3]:
# Transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [4]:
# Setting Library
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd

In [5]:
device = torch.device("cuda:0") 

In [6]:
# BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

using cached model. /DaiS_internship/DongSung/temp/.cache/kobert_v1.zip
using cached model. /DaiS_internship/DongSung/temp/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [7]:
data = pd.read_csv('db/UserRestReview.csv')

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40225 entries, 0 to 40224
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   r_id    40225 non-null  int64 
 1   u_id    40225 non-null  int64 
 2   p_id    40225 non-null  int64 
 3   rating  40225 non-null  int64 
 4   review  40225 non-null  object
dtypes: int64(4), object(1)
memory usage: 1.5+ MB


In [9]:
data['class'] = [1 if s >=3 else 0 for s in data['rating']] # 긍정이면 1, 부정이면 0
data.drop_duplicates(subset = ['review'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
data['review'] = data['review'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
data['review'] = data['review'].str.replace('^ +', "") # 공백은 empty 값으로 변경
data['review'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
data = data.dropna(how='any') # Null 값 제거

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35268 entries, 0 to 40224
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   r_id    35268 non-null  int64 
 1   u_id    35268 non-null  int64 
 2   p_id    35268 non-null  int64 
 3   rating  35268 non-null  int64 
 4   review  35268 non-null  object
 5   class   35268 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 1.9+ MB


In [11]:
data_list = []
for q, label in zip(data['review'], data['class'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [12]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.2, shuffle = True, random_state = 23)

In [13]:
print(len(dataset_train), len(dataset_test))

28214 7054


In [14]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

using cached model. /DaiS_internship/DongSung/temp/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [15]:
# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
# 출처 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab = vocab, pad = pad, pair = pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [16]:
max_len = 100
batch_size = 64
warmup_ratio = 0.1
num_epochs = 7
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [17]:
# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [18]:
# torch 형식의 dataset을 만들어 입력 데이터셋의 전처리 마무리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)


In [19]:
# KoBERT 오픈소스 내 예제코드 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 2,   # 감정 클래스 수로 조정 긍정/부정 클래스2개
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [20]:
# BERT  모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

In [21]:
# optimizer와 schedule 설정
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)


In [22]:
# calc_accuracy : 정확도 측정을 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [23]:
train_history = []
test_history = []
loss_history = []

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        # print(label.shape, out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    # train_history.append(train_acc / (batch_id+1))

    # .eval() : nn.Module에서 train time과 eval time에서 수행하는 다른 작업을 수행할 수 있도록 switching 하는 함수
    # 즉, model이 Dropout이나 BatNorm2d를 사용하는 경우, train 시에는 사용하지만 evaluation을 할 때에는 사용하지 않도록 설정해주는 함수
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))


  0%|          | 0/441 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.6686797142028809 train acc 0.59375
epoch 1 batch id 201 loss 0.2450888752937317 train acc 0.8746113184079602
epoch 1 batch id 401 loss 0.20817868411540985 train acc 0.896158042394015
epoch 1 train acc 0.8990089863105737


  0%|          | 0/111 [00:00<?, ?it/s]

epoch 1 test acc 0.9322916666666666


  0%|          | 0/441 [00:00<?, ?it/s]

...

epoch 6 test acc 0.9348254504504504


  0%|          | 0/441 [00:00<?, ?it/s]

epoch 7 batch id 1 loss 0.0018172896234318614 train acc 1.0
epoch 7 batch id 201 loss 0.028371430933475494 train acc 0.9933146766169154
epoch 7 batch id 401 loss 0.009639808908104897 train acc 0.9941162718204489
epoch 7 train acc 0.9943310657596371


  0%|          | 0/111 [00:00<?, ?it/s]

epoch 7 test acc 0.9351069819819819


In [24]:
def predict(predict_sentence): # input = 감정분류하고자 하는 sentence

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False) # 토큰화한 문장
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size = batch_size, num_workers = 5) # torch 형식 변환
    
    model.eval() 

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval = []
        for i in out: # out = model(token_ids, valid_length, segment_ids)
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("부정")
            else:
                test_eval.append("긍정")

        print("해당 리뷰는" + test_eval[0] + "입니다.")

In [25]:
# 질문에 0 입력 시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

해당 리뷰는부정입니다.


해당 리뷰는긍정입니다.


해당 리뷰는부정입니다.


해당 리뷰는긍정입니다.


해당 리뷰는긍정입니다.


해당 리뷰는긍정입니다.


해당 리뷰는부정입니다.




In [41]:
import pathlib
import os
MODEL_PATH = os.getcwd() + '/model_save'
os.makedirs(MODEL_PATH, exist_ok=True)

In [44]:
torch.save(model.state_dict(), MODEL_PATH + '/kobert.pt')